In [1]:
from google.colab import drive
import os
drive.mount('/content/drive/',force_remount=True)
os.chdir("/content/drive/MyDrive/Colab Notebooks/LearningToCountEverything")
os.listdir()

Mounted at /content/drive/


['test.py',
 'README.md',
 'utils.py',
 'model.py',
 'logsSave',
 '__pycache__',
 'data',
 '.git',
 'train.py',
 'utils_DO_NOT_TOUCH.ipynb',
 'AML-LTCE-Train.ipynb',
 'AML-LTCE-Test.ipynb']

In [2]:
import copy
from model import CountRegressor, Resnet50FPN
from utils import MAPS, Scales, Transform, extract_features
from utils import MincountLoss, PerturbationLoss
from PIL import Image
import os
import torch
import argparse
import json
import numpy as np
from tqdm import tqdm
from os.path import exists
import torch.optim as optim

In [ ]:
"""parser = argparse.ArgumentParser(description="Few Shot Counting Evaluation code")
parser.add_argument("-dp", "--data_path", type=str, default='./data/', help="Path to the FSC147 dataset")
parser.add_argument("-ts", "--test_split", type=str, default='val', choices=["val_PartA","val_PartB","test_PartA","test_PartB","test", "val"], help="what data split to evaluate on")
parser.add_argument("-m",  "--model_path", type=str, default="./data/pretrainedModels/FamNet_Save1.pth", help="path to trained model")
parser.add_argument("-a",  "--adapt", action='store_true', help="If specified, perform test time adaptation")
parser.add_argument("-gs", "--gradient_steps", type=int,default=100, help="number of gradient steps for the adaptation")
parser.add_argument("-lr", "--learning_rate", type=float,default=1e-7, help="learning rate for adaptation")
parser.add_argument("-wm", "--weight_mincount", type=float,default=1e-9, help="weight multiplier for Mincount Loss")
parser.add_argument("-wp", "--weight_perturbation", type=float,default=1e-4, help="weight multiplier for Perturbation Loss")
parser.add_argument("-g",  "--gpu-id", type=int, default=0, help="GPU id. Default 0 for the first GPU. Use -1 for CPU.")
args = parser.parse_args()"""


In [4]:
data_path = "./data/"
test_split = "val" #choices=["val_PartA","val_PartB","test_PartA","test_PartB","test", "val"]
model_path = "./data/pretrainedModels/FamNet_Save1.pth"
adapt = "store_true"
gradient_steps = 100
learning_rate  = 1e-7
weight_mincount = 1e-9
weight_perturbation = 1e-4
gpu_id = 0 

#data_path = args.data_path
anno_file = data_path + 'annotation_FSC147_384.json'
data_split_file = data_path + 'Train_Test_Val_FSC_147.json'
im_dir = data_path + 'images_384_VarV2'

In [5]:
if not exists(anno_file) or not exists(im_dir):
    print("Make sure you set up the --data-path correctly.")
    print("Current setting is {}, but the image dir and annotation file do not exist.".format(data_path))
    print("Aborting the evaluation")
    exit(-1)

if not torch.cuda.is_available() or gpu_id < 0:
    use_gpu = False
    print("===> Using CPU mode.")
else:
    use_gpu = True
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

In [6]:
resnet50_conv = Resnet50FPN()
if use_gpu: resnet50_conv.cuda()
resnet50_conv.eval()

regressor = CountRegressor(6, pool='mean')
regressor.load_state_dict(torch.load(model_path))
if use_gpu: regressor.cuda()
regressor.eval()

with open(anno_file) as f:
    annotations = json.load(f)

with open(data_split_file) as f:
    data_split = json.load(f)




Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
cnt = 0
SAE = 0  # sum of absolute errors
SSE = 0  # sum of square errors

print("Evaluation on {} data".format(test_split))
im_ids = data_split[test_split]
pbar = tqdm(im_ids)
for im_id in pbar:
    anno = annotations[im_id]
    bboxes = anno['box_examples_coordinates']
    dots = np.array(anno['points'])

    rects = list()
    for bbox in bboxes:
        x1, y1 = bbox[0][0], bbox[0][1]
        x2, y2 = bbox[2][0], bbox[2][1]
        rects.append([y1, x1, y2, x2])

    image = Image.open('{}/{}'.format(im_dir, im_id))
    image.load()
    sample = {'image': image, 'lines_boxes': rects}
    sample = Transform(sample)
    image, boxes = sample['image'], sample['boxes']

    if use_gpu:
        image = image.cuda()
        boxes = boxes.cuda()

    with torch.no_grad(): features = extract_features(resnet50_conv, image.unsqueeze(0), boxes.unsqueeze(0), MAPS, Scales)

    if not adapt:
        with torch.no_grad(): output = regressor(features)
    else:
        features.required_grad = True
        adapted_regressor = copy.deepcopy(regressor)
        adapted_regressor.train()
        optimizer = optim.Adam(adapted_regressor.parameters(), lr=learning_rate)
        for step in range(0, gradient_steps):
            optimizer.zero_grad()
            output = adapted_regressor(features)
            lCount = weight_mincount * MincountLoss(output, boxes)
            lPerturbation = weight_perturbation * PerturbationLoss(output, boxes, sigma=8)
            Loss = lCount + lPerturbation
            # loss can become zero in some cases, where loss is a 0 valued scalar and not a tensor
            # So Perform gradient descent only for non zero cases
            if torch.is_tensor(Loss):
                Loss.backward()
                optimizer.step()
        features.required_grad = False
        output = adapted_regressor(features)

    gt_cnt = dots.shape[0]
    pred_cnt = output.sum().item()
    cnt = cnt + 1
    err = abs(gt_cnt - pred_cnt)
    SAE += err
    SSE += err**2

    pbar.set_description('{:<8}: actual-predicted: {:6d}, {:6.1f}, error: {:6.1f}. Current MAE: {:5.2f}, RMSE: {:5.2f}'.\
                         format(im_id, gt_cnt, pred_cnt, abs(pred_cnt - gt_cnt), SAE/cnt, (SSE/cnt)**0.5))
    print("")

print('On {} data, MAE: {:6.2f}, RMSE: {:6.2f}'.format(args.test_split, SAE/cnt, (SSE/cnt)**0.5))

Evaluation on val data


  0%|          | 0/1286 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
190.jpg : actual-predicted:     13,   58.7, error:   45.7. Current MAE: 45.69, RMSE: 45.69:   0%|          | 1/1286 [01:00<21:41:36, 60.78s/it]

/content/drive/My Drive/Colab Notebooks/LearningToCountEverything/utils.py:99: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  Loss += F.mse_loss(X,ones)
191.jpg : actual-predicted:     15,   17.0, error:    2.0. Current MAE: 23.86, RMSE: 32.34:   0%|          | 2/1286 [01:17<12:27:27, 34.93s/it]

192.jpg : actual-predicted:     19,   17.0, error:    2.0. Current MAE: 16.57, RMSE: 26.43:   0%|          | 3/1286 [01:39<10:16:23, 28.83s/it]

194.jpg : actual-predicted:     82,   61.8, error:   20.2. Current MAE: 17.48, RMSE: 25.02:   0%|          | 4/1286 [02:00<9:12:32, 25.86s/it] 

...

229.jpg : actual-predicted:     25,   46.9, error:   21.9. Current MAE: 21.65, RMSE: 38.79:   2%|▏         | 22/1286 [07:52<7:15:49, 20.69s/it]

231.jpg : actual-predicted:     24,   29.8, error:    5.8. Current MAE: 20.96, RMSE: 37.96:   2%|▏         | 23/1286 [08:13<7:13:32, 20.60s/it]

232.jpg : actual-predicted:     27,   46.8, error:   19.8. Current MAE: 20.91, RMSE: 37.38:   2%|▏         | 24/1286 [08:34<7:20:08, 20.93s/it]

233.jpg : actual-predicted:    126,  108.0, error:   18.0. Current MAE: 20.79, RMSE: 36.80:   2%|▏         | 25/1286 [08:56<7:24:44, 21.16s/it]

234.jpg : actual-predicted:     38,   42.2, error:    4.2. Current MAE: 20.15, RMSE: 36.09:   2%|▏         | 26/1286 [09:22<7:54:48, 22.61s/it]